# erf_table_draft2017b.ipynb

## Purpose
Calculate area-weighted mean global radiative effects for *"Effective radiative forcing in the aerosol-climate model CAM5.3-MARC-ARG"*, based on results from project [p17c-marc-comparison](https://github.com/grandey/p17c-marc-comparison).

## Requirements
- NetCDF files produced by [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb).
- Python modules mentioned in cell #2, inlcuding [climapy](https://github.com/grandey/climapy) ([doi:10.5281/zenodo.1053020](https://doi.org/10.5281/zenodo.1053020)).

## Author
Benjamin S. Grandey, 2018

In [1]:
! date

In [2]:
import climapy
import os
import numpy as np
import xarray as xr

# Print versions of packages
for module in [climapy, np, xr]:
    try:
        print('{}.__version__ = {}'.format(module.__name__, module.__version__))
    except AttributeError:
        pass

climapy.__version__ = 0.1.0+b35937c
numpy.__version__ = 1.14.3
xarray.__version__ = 0.10.4


## Post-processed data file location
Location of files written by [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb).

In [3]:
in_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/analysis_cdo_nco_draft2017b/')

## Read global means and standard errors into dictionaries

In [4]:
model_var_dict_list = [
    {'mam3': 'cDRE', 'mam7': 'cDRE', 'marc_s2': 'cDRE', },
    {'mam3': 'SWCF_d1', 'mam7': 'SWCF_d1', 'marc_s2': 'CRF'},
    {'mam3': 'LWCF', 'mam7': 'LWCF', 'marc_s2': 'LWCF'},
    {'mam3': 'FSNTOAC_d1', 'mam7': 'FSNTOAC_d1', 'marc_s2': 'FSNTOACNOA'},
    {'mam3': 'cFNTOA', 'mam7': 'cFNTOA', 'marc_s2': 'cFNTOA'}]

In [5]:
# Initialise dictionaries
mean_dict = {}  # mean global totals
error_dict = {}  # standard errors, based on annual means
# Loop over model, variable combinations
for model_var_dict in model_var_dict_list:
    for model, variable in model_var_dict.items():
        # Loop over years
        for year in ['2000', '1850']:
            model_var_year = (model, variable, year)
            # Read input data
            in_filename = '{}/{}_{}_{}_ANN.nc'.format(in_dir, model, year, variable)
            in_ds = xr.open_dataset(in_filename, decode_times=False)[variable]
            # Exclude first two years as spin-up
            data = in_ds.isel(time=slice(2, None))
            # Calculate area-weighted mean
            totals = climapy.xr_area_weighted_stat(data, stat='mean')
            # Calculate mean across years
            mean_dict[model_var_year] = float(totals.mean())
            # Calculate standard error
            error = totals.std(dim='time', ddof=1).data.flatten() / np.sqrt(totals['time'].size)
            error_dict[model_var_year] = float(error)
            # Close input file
            in_ds.close()
        # Calculate 2000-1850 diff
        mean_2000 = mean_dict[(model, variable, '2000')]
        mean_1850 = mean_dict[(model, variable, '1850')]
        error_2000 = error_dict[(model, variable, '2000')]
        error_1850 = error_dict[(model, variable, '1850')]
        mean_diff = mean_2000 - mean_1850
        error_diff = np.sqrt(error_2000**2 + error_1850**2)
        mean_dict[(model, variable, '2000-1850')] = mean_diff
        error_dict[(model, variable, '2000-1850')] = error_diff
        # Print result
        print('{:7}, {:10}: {:+6.2f} ± {:4.2f}'.format(model, variable, mean_diff, error_diff))
!date

mam3   , cDRE      :  -0.02 ± 0.01
mam7   , cDRE      :  -0.00 ± 0.01
marc_s2, cDRE      :  -0.17 ± 0.01
mam3   , SWCF_d1   :  -2.09 ± 0.04
mam7   , SWCF_d1   :  -2.05 ± 0.04
marc_s2, CRF       :  -2.17 ± 0.04
mam3   , LWCF      :  +0.54 ± 0.02
mam7   , LWCF      :  +0.53 ± 0.02
marc_s2, LWCF      :  +0.66 ± 0.01
mam3   , FSNTOAC_d1:  +0.00 ± 0.02
mam7   , FSNTOAC_d1:  -0.02 ± 0.02
marc_s2, FSNTOACNOA:  -0.10 ± 0.02
mam3   , cFNTOA    :  -1.57 ± 0.04
mam7   , cFNTOA    :  -1.53 ± 0.04
marc_s2, cFNTOA    :  -1.79 ± 0.03
Mon Jul 30 16:26:41 +08 2018


Values above can be copied in Table 5 of manuscript.